This script allows to filter rasters to test some result and functions.

- based on pixel values so we can visualize the extrem pixels and look out for anomaly
- to test the mask function
- to produce attention mask from polygon and extend

In [ ]:
import os, sys
import logging, logging.config
import yaml
import glob
from joblib import Parallel, delayed
from tqdm import tqdm

import pandas as pd
import geopandas as gpd
from shapely.geometry import shape
from shapely.geometry import mapping, Point, Polygon

import rasterio
from rasterio.mask import mask
from rasterio.features import shapes, rasterize

import numpy as np
import matplotlib.pyplot as plt


import fct_misc

# from helpers import XYZ


In [ ]:
TILES_DIR='/mnt/data-01/gsalamin/proj-roadsurf-b/02_Data/processed/obj_detector/oth-images'
ROADS='/mnt/data-01/gsalamin/proj-roadsurf-b/02_Data/processed/shapefiles_gpkg/roads_polygons.shp'
TILES_INFO='/mnt/data-01/gsalamin/proj-roadsurf-b/02_Data/processed/json/tiles_aoi.geojson'

shp_gpkg_folder='/mnt/data-01/gsalamin/proj-roadsurf-b/02_Data/processed/shapefiles_gpkg/'

roads=gpd.read_file(ROADS)
tiles_info=gpd.read_file(TILES_INFO)

# Testing extrem values
## Making polygons on the zones to check

In [ ]:
files=glob.glob(TILES_DIR+'/*.tif')
print(files[:2])

In [ ]:
geom=[]
bands=[]
pixel_values=[]

for file in tqdm(files, desc='Checking files'):
    for band in range(1,5):
        with rasterio.open(file) as f:
            image = f.read(band)

            lim_sup=200
            lim_inf=1

            # create a binary image, 0 where there's nodata, 1 where it's valid
            is_valid = ((image < lim_inf) | (image > lim_sup)).astype(np.uint8)

            
            # vectorize the binary image, supplying the transform so it returns maps coords
            for coords, value in shapes(is_valid, transform=f.transform):

                # ignore polygons corresponding to nodata
                if value != 0:
                    # convert geojson to shapely geometry
                    geom.append(shape(coords))
                    bands.append(band)
                    pixel_values.append(value)

fid=[x for x in range(1, len(geom)+1)]
zones_dict={'fid':fid, 'band':bands, 'pixel_value':pixel_values, 'geometry': geom}


In [ ]:
extrem_zones=gpd.GeoDataFrame(zones_dict, crs='EPSG:3857')

In [ ]:
roads_reproject=roads.to_crs(epsg=3857)

misc_fct.test_crs(roads_reproject.crs, extrem_zones.crs)

extrem_zones_on_roads=gpd.overlay(extrem_zones,roads_reproject[['OBJECTID', 'geometry']])

In [ ]:
extrem_zones_on_roads.shape

In [ ]:
extrem_zones_on_roads.drop_duplicates(subset=['fid'], inplace=True, ignore_index=True)

extrem_zones_on_roads.to_file('/mnt/data-01/gsalamin/proj-roadsurf-b/02_Data/processed/shapefiles_gpkg/test_extrem_pixels.shp')

In [ ]:
extrem_zones_on_roads.shape

## Downloading tiles for the zones to check

In [ ]:
with open('config.yaml') as fp:
    cfg = yaml.load(fp, Loader=yaml.FullLoader)['generate_tilesets.py']

OUTPUT_DIR = '/mnt/data-01/gsalamin/proj-roadsurf-b/02_Data/processed/images'

ORTHO_WS_TYPE = cfg['datasets']['orthophotos_web_service']['type']
ORTHO_WS_URL = cfg['datasets']['orthophotos_web_service']['url']
ORTHO_WS_SRS = cfg['datasets']['orthophotos_web_service']['srs']
if 'layers' in cfg['datasets']['orthophotos_web_service'].keys():
    ORTHO_WS_LAYERS = cfg['datasets']['orthophotos_web_service']['layers']
if 'parameters' in cfg['datasets']['orthophotos_web_service'].keys():
    ORTHO_WS_PARAMETERS=cfg['datasets']['orthophotos_web_service']['parameters']
else:
    ORTHO_WS_PARAMETERS={}

SAVE_METADATA = True
OVERWRITE = cfg['overwrite']
TILE_SIZE = cfg['tile_size']

ALL_IMG_PATH = os.path.join(OUTPUT_DIR, f"test")
if not os.path.exists(ALL_IMG_PATH):
        os.makedirs(ALL_IMG_PATH)

In [ ]:
tiles_info_reproj=tiles_info.to_crs(crs=3857)

fct_misc.test_crs(tiles_info_reproj.crs, extrem_zones_on_roads.crs)

tiles_info_on_zones=gpd.overlay(tiles_info_reproj, extrem_zones_on_roads[['fid','geometry']])

In [ ]:
tiles_info_on_zones.drop_duplicates(subset=['id'], inplace=True, ignore_index=True)

In [ ]:
job_dict = XYZ.get_job_dict(
    tiles_gdf=tiles_info_on_zones.to_crs(ORTHO_WS_SRS), # <- note the reprojection
    XYZ_url=ORTHO_WS_URL, 
    img_path=ALL_IMG_PATH, 
    save_metadata=SAVE_METADATA,
    overwrite=OVERWRITE
)

image_getter = XYZ.get_geotiff

In [ ]:
import warnings

with warnings.catch_warnings(record=True):
    N_JOBS=10
    job_outcome = Parallel(n_jobs=N_JOBS, backend="loky")(
                delayed(image_getter)(**v) for k, v in tqdm( sorted(list(job_dict.items())) )
        )

    all_tiles_were_downloaded = True
    for job in job_dict.keys():
        if not os.path.isfile(job) or not os.path.isfile(job.replace('.tif', '.json')):
            all_tiles_were_downloaded = False
            print('Failed task: ', job)

    if all_tiles_were_downloaded:
        print("...done.")
    else:
        print("Some tiles were not downloaded. Please try to run this script again.")
        sys.exit(1)

# Testing the mask 

In [ ]:
from rasterio import features

In [ ]:
def burn_mask(src_img_filename, dst_img_filename, polys):

    with rasterio.open(src_img_filename) as src:

        src_img = src.read(4)

        if polys == []:
            # TODO: check whether we should replace the following with mask = None
            mask = src_img != -1 # -> everywhere
        else:

            mask = features.geometry_mask(polys, 
                                          out_shape=src.shape, 
                                          transform=src.transform,
                                          all_touched=True)
        shapes = features.shapes(src_img, 
                                 mask=mask, 
                                 transform=src.transform)
        
        profile = src.profile
        profile.update(dtype=rasterio.uint8, count=1)
        
        image = features.rasterize(((g, 255) for g, v in shapes), 
                                   out_shape=src.shape, 
                                   transform=src.transform)
    
    with rasterio.open(dst_img_filename, 'w', **profile) as dst:
        dst.write(image, indexes=1)
    
    return


In [ ]:
if roads[roads.is_valid==False].shape[0]!=0:
        print(f"There are {roads[roads.is_valid==False].shape[0]} invalid geometries for the roads.")
        sys.exit(1)          

simplified_roads=roads.drop(columns=['ERSTELLUNG', 'ERSTELLU_1', 'HERKUNFT', 'HERKUNFT_J', 'HERKUNFT_M',
        'KUNSTBAUTE', 'WANDERWEGE', 'VERKEHRSBE', 'BEFAHRBARK', 'EROEFFNUNG', 'STUFE', 'RICHTUNGSG',
        'KREISEL', 'EIGENTUEME', 'VERKEHRS_1', 'NAME', 'TLM_STRASS', 'STRASSENNA', 'SHAPE_Leng'])

roads_reproj=simplified_roads.to_crs(epsg=3857)
tiles_info_reproj=tiles_info.to_crs(epsg=3857)

fp_list=[]
name_list=[]
for tile_idx in tiles_info_reproj['id'].values:
        # Get the name of the tiles
        x, y, z = tile_idx.lstrip('(,)').rstrip('(,)').split(',')
        im_name = z.lstrip() + '_' + x + '_' + y.lstrip() + '.tif'
        im_path = os.path.join(TILES_DIR, im_name)
        fp_list.append(im_path)
        name_list.append(im_name)

tiles_info_reproj['filepath']=fp_list
tiles_info_reproj['filename']=name_list

fct_misc.test_crs(roads_reproj.crs, tiles_info_reproj.crs)

if roads_reproj[roads_reproj.is_valid==False].shape[0]!=0:
        print(f"There are {roads_reproj[roads_reproj.is_valid==False].shape[0]} invalid geometries for the roads after the reprojection.")

print("Correction of the roads presenting an invalid geometry with a buffer of 0 m...")
corrected_roads=roads_reproj.copy()
corrected_roads.loc[corrected_roads.is_valid==False,'geometry']=corrected_roads[corrected_roads.is_valid==False]['geometry'].buffer(0)


In [ ]:
fct_misc.test_crs(corrected_roads.crs, tiles_info_reproj.crs)
intersected_tiles=gpd.sjoin(tiles_info_reproj, corrected_roads[['OBJECTID', 'geometry']])
intersected_tiles.drop_duplicates(subset=['id','OBJECTID'], inplace=True)

try:
    assert not corrected_roads['OBJECTID'].duplicated().any()
except:
    print('Some roads are separated on mulitple lines. They must be transformed to multipolygons or fused first.')
    sys.exit(1)

pixels_per_band=pd.DataFrame()

In [ ]:
BANDS=range(1,5)

17_68352_46260.tif  
17_68373_46267.tif

Depuis le tableau intersected tiles, choisir les routes qui correspondent à ces tuiles

In [ ]:
# %%timeit

pixels_per_band=pd.DataFrame()
errors=pd.DataFrame(columns=['filename', 'band', 'datashape', 'nbr_of_pix'])
it=0

for road in corrected_roads.itertuples():

    objectid=road.OBJECTID

    # Get the corresponding tile(s)
    intersected_tiles_with_road=intersected_tiles[intersected_tiles['OBJECTID'] == objectid].copy()
    intersected_tiles_with_road.reset_index(drop=True, inplace=True)

    if intersected_tiles_with_road.shape[0] == 0:
        continue

    # Get the pixels for each tile
    for tile_filepath in intersected_tiles_with_road['filepath'].values:
        pixel_values=pd.DataFrame()
        
        # extract the geometry in GeoJSON format
        geoms = [mapping(road.geometry)]

        # extract the raster values values within the polygon 
        with rasterio.open(tile_filepath) as src:
            out_image, _ = mask(src, geoms, crop=True, filled=False)

            # no data values of the original raster
            no_data=src.nodata

        dico={}
        length_bands=[]
        for band in BANDS:

            # extract the values of the masked array
            data = out_image[band-1]

            # extract the the valid values
            val = np.extract(~data.mask, data.data)

            dico[f'band{band}']=val
            length_bands.append(len(val))

        dico.update({'road_id': objectid})

        try:
            pixels_from_tile = pd.DataFrame(dico)
        except ValueError:
            for band in BANDS:
                data = out_image[band-1]

                errors.loc[len(errors.index)]=[tile_filepath[-18:], band, data.data.shape, (~data.mask).sum()]

            # pd.DataFrame(~data.mask).to_csv(f"~/Downloads/{tile_filepath[-18:-4]}.csv", index=False)
            directory = "/mnt/data-01/gsalamin/proj-roadsurf-b/02_Data/processed/images/test2"
            burn_mask(tile_filepath, os.path.join(directory, f"{tile_filepath[-18:-4]}_{it}.tif"), geoms)
            it+=1
            continue

        pixel_values = pd.concat([pixel_values, pixels_from_tile],ignore_index=True)

    pixels_per_band=pd.concat([pixels_per_band, pixel_values], ignore_index=True)

In [ ]:
errors.to_csv('~/Downloads/uneven_pixels.csv', index=False)

# Produce attention mask from polygon and raster
The idea is to produce an attention mask to use as the 4th or 5th band of a raster

cf. https://lpsmlgeo.github.io/2019-09-22-binary_mask/
cf. https://rasterio.readthedocs.io/en/latest/api/rasterio.features.html

In [ ]:
def poly_from_utm(polygon, transform):
    poly_pts = []
    
    for i in np.array(polygon.exterior.coords):
        
        # Convert polygons to the image CRS
        poly_pts.append(~transform * tuple(i))
        
    # Generate a polygon object
    new_poly = Polygon(poly_pts)
    return new_poly

In [ ]:
ROADS='/mnt/data-01/gsalamin/proj-roadsurf-b/02_Data/processed/shapefiles_gpkg/roads_MO_TLM.gpkg'
ROADS_LAYER="sectionned_roads_by_surface"
TILES="/mnt/data-01/gsalamin/proj-roadsurf-b/02_Data/processed/json/tiles_aoi.geojson"
IMAGES_DIR="/mnt/data-01/gsalamin/proj-roadsurf-b/02_Data/processed/obj_detector/all-images"

In [ ]:
roads=gpd.read_file(ROADS, layer=ROADS_LAYER)
tiles=gpd.read_file(TILES)

In [ ]:
print([tile.lstrip('(').rstrip(')').split(', ') for tile in tiles[0:3]['id']])

In [ ]:
roads_union_geom=roads.unary_union
roads_union=gpd.GeoDataFrame({'id_roadset': [i for i in range(len(roads_union_geom.geoms))],
                            'geometry': [geo for geo in roads_union_geom.geoms]},
                            crs=roads.crs
                            )

In [ ]:
fct_misc.test_crs(tiles.crs, roads_union.crs)
inv_masks_tiles=gpd.overlay(tiles, roads_union, how="difference")
# inv_masks.to_file(os.path.join(shp_gpkg_folder, 'test_mask.shp'))

In [ ]:
inv_masks_tiles_3857=inv_masks_tiles.to_crs(epsg=3857)

# cf. https://lpsmlgeo.github.io/2019-09-22-binary_mask/
for tile_row in inv_masks_tiles_3857[:1].itertuples():
    tile_id=tile_row.id

    # Get the tile filename
    x, y, z = tile_id.lstrip('(').rstrip(')').split(', ')
    filename=z+'_'+x+'_'+y+'.tif'

    # Get the tile
    with rasterio.open(os.path.join(IMAGES_DIR, filename), "r") as src:
        tile_img = src.read()
        tile_meta = src.meta
    
    # fct_misc.test_crs(tile_meta['crs'], inv_masks_tiles.crs)

    im_size = (tile_meta['height'], tile_meta['width'])

    inv_mask = rasterize(shapes=[poly_from_utm(geom, src.meta['transform']) for geom in tile_row.geometry.geoms],
                    out_shape=im_size)
                    
    tile_mask = (1-inv_mask) * 255

    tile_img_augmented=np.ndarray(shape=(5,256,256), dtype='uint8')
    tile_img_augmented[0:4,:,:]=tile_img
    tile_img_augmented[4,:,:]=tile_mask


In [ ]:
mask_meta = src.meta.copy()
mask_meta.update({'count': tile_img_augmented.shape[0]+1})
with rasterio.open(os.path.join('/home/gsalamin/Downloads', 'test.tif'), 'w', **mask_meta) as dst:
    dst.write(tile_img_augmented, [band for band in range(1,tile_img_augmented.shape[0]+1)])

In [ ]:
[band for band in range(1,tile_img_augmented.shape[0]+1)]

In [ ]:
tile_meta

In [ ]:
plt.figure(figsize=(15,15))
plt.imshow(tile_mask)